<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 5 - preprocessing i postprocessing

Na dzisiejszych zajęciach zajmiemy się niezwykle przydatnymi narzędziami wspomagającymi pracę tłumacza. W odróżnieniu od dotychczas poznanych, nie są one oparte na pamięci tłumaczeń, ani na słownikach. Chodzi o techniki preprocessingu i postprocessingu.

Proces tłumaczenia przeprowadzony w pełni profesjonalnie składa się z wielu faz, które angażują nie tylko tłumaczy, ale także kierowników projektu, analityków, czy korektorów. Każda z tych osób do swojej pracy może wykorzystywać system informatyczny, do którego na początku całego procesu trafiają pliki do tłumaczenia. Oznacza to, że zanim tekst źródłowy trafi do tłumacza, system ma jeszcze szansę coś w nim zmienić. A kiedy tłumacz wykona już swoją pracę, można uruchomić kolejny mechanizm, który zmodyfikuje tłumaczenie przed oddaniem go do zamawiającego. Jak się domyślamy, modyfikacje tekstu przed przekazaniem go do tłumacza nazywamy **preprocessingiem**, natomiast te dokonywane po wykonaniu tłumaczenia (ale przed zwróceniem go do klienta) nazywamy **postprocessingiem**. Terminy te, będące mało zgrabnymi kalkami z języka angielskiego, mają wersje prawdziwie polskie: przetwarzanie wstępne i końcowe. Wersje te są jednak stosowane na tyle rzadko, że mogą jedynie wprowadzić zamieszanie (co w gruncie rzeczy jest dość smutne).

Typowe operacje w fazie preprocessingu obejmują:
* identyfikację tagów xmlowych (które często są później wyświetlane w interfejsie CAT-a jako jeden niepodzielny znak)
* identyfikację segmentów, których nie należy tłumaczyć (na przykład składających się z samych liczb)

* identyfikację dat i jednostek miary w tekście źródłowym

We wszystkich tych operacjach niezwykle przydatne okazują się wyrażenia regularne.

### Ćwiczenie 1: Używając wyrażeń regularnych napisz funkcję do znajdowania wszystkich tagów XML w tekście. Funkcja powinna zwracać pozycje, na których znalazła tagi.

In [4]:
import re

def find_tags(text):
    return [(tag.start(), tag.end()) for tag in list(re.finditer('(</?[0-9A-z]+>)', text))]

find_tags('<title>Tytuł</title>')

[(0, 7), (12, 20)]

### Ćwiczenie 2: Używając wyrażeń regularnych napisz funkcję do identyfikacji segmentów, których nie należy tłumaczyć. Zastosuj wymyślone przez siebie kryteria.

In [5]:
def is_translatable(text):
    for pattern in ['^-?[0-9][0-9,\.]+$', 'www.', 'https?://']:
        if re.match(pattern, text) is not None:
            return False
    return True

is_translatable('https://pl.wikipedia.org/wiki/Pa%C5%84stwa_uczestnicz%C4%85ce_w_Zimowych_Igrzyskach_Olimpijskich_1988')

False

### Ćwiczenie 3: Używając wyrażeń regularnych napisz funkcję do identyfikacji i interpretacji 5 wybranych przez siebie formatów daty. Funkcja powinna zwracać pozycje, na których odnalazła daty oraz dzień, miesiąc i rok, które ta data reprezentuje.

In [6]:
def retrieve_data(match, format):
    val = match.group()
    pos_start = match.start()
    pos_end = match.end()
    if (format == 'rmd'): # te nazwy formatów moje własne
        year = val[:4]
        month = val[5:7]
        day = val[8:]
    elif (format == 'dmr'):
        year = val[6:]
        month = val[3:5]
        day = val[:2]
    elif (format == 'dmr-short'):
        parts = val.split('/')
        year = parts[2]
        month = parts[1]
        day = parts[0]
    elif (format == 'american?'):
        parts = val.split('/')
        year = parts[2]
        month = parts[0]
        day = parts[1]
    return {
        'pos_start': pos_start,
        'pos_end': pos_end,
        'day': day,
        'month': month,
        'year': year
    }


def find_dates(text):
    patterns = [
        ('[0-9]{4}-(0[0-9]|1[1-2])-([0-2][0-9]|3[0-1])', 'rmd'),
        ('[0-9]{4}/(0[0-9]|1[1-2])/([0-2][0-9]|3[0-1])', 'rmd'),
        ('([0-2][0-9]|3[0-1])\.(0[1-9]|1[1-2])\.[0-9]{4}', 'dmr'),
        ('([1-9]|[1-2][0-9]|3[0-1])/([0-9]|1[1-2])/[0-9]{4}', 'dmr-short'),
        ('(0[1-9]|1[1-2])/([0-2][0-9]|3[0-1])/[0-9]{4}', 'american?')
    ]
    dates = []
    for (pattern, format) in patterns:
        matches = list(re.finditer(pattern, text))
        for match in matches:
            if match is not None:
                dates.append(retrieve_data(match, format))
    return dates

find_dates('2021-04-07 2021/04/07 07.04.2021 7/4/2021 04/07/2021' )


[{'day': '07', 'month': '04', 'pos_end': 10, 'pos_start': 0, 'year': '2021'},
 {'day': '07', 'month': '04', 'pos_end': 21, 'pos_start': 11, 'year': '2021'},
 {'day': '07', 'month': '04', 'pos_end': 32, 'pos_start': 22, 'year': '2021'},
 {'day': '7', 'month': '4', 'pos_end': 41, 'pos_start': 33, 'year': '2021'},
 {'day': '07', 'month': '04', 'pos_end': 52, 'pos_start': 42, 'year': '2021'}]

Po preprocessingu i tłumaczeniu czas na postprocessing. Ponieważ wykonywany jest on na przetłumaczonym tekście, jego głównym zadaniem jest eliminacja błędów popełnionych przez tłumacza w fazie tłumaczenia. Podczas postprocessingu najczęściej wykonuje się:
* korektę pisowni dla języka docelowego
* usuwanie błędów typograficznych z tekstu (np. wielokrotne spacje, brak spacji po przecinku)
Stanowi to bardzo ważne wsparcie dla edytorów i korektorów, czyli osób sprawdzających pracę tłumacza.

Jednak nowoczesne CAT-y potrafią coś jeszcze. Są w stanie w sprytny sposób wykorzystać kombinację pre- i postprocessingu do wyręczenia tłumacza w żmudnych i technicznych czynnościach. Wykonajmy następujące ćwiczenie:

### Ćwiczenie 4: Wykorzystując funkcję find_dates napisz funkcję do obsługi dat w tłumaczeniu. Wejściem jest segment źródłowy oraz docelowy, które zawierają daty, przy czym daty te mogą być w różnych formatach. Dodatkowym parametrem wejściowym jest nazwa oczekiwanego formatu daty w tłumaczeniu (np. "Europe", "US", "digit-dot". Funkcja najpierw sprawdza, czy liczba dat w tłumaczeniu zgadza się z liczbą dat w segmencie źródłowym oraz czy odpowiadające sobie daty wskazują na ten sam dzień. Jeśli nie, wypisywane jest stosowne ostrzeżenie. Oczekiwanym wyjściem jest segment docelowy, w którym wszystkie daty są w żądanym formacie. 

In [68]:
def build_date(date, date_format):
    if date_format == 'rrrr-mm-dd':
        return date['year'] + '-' + str(date['month'])  + '-' + str(date['day'])
    if date_format == 'dd.mm.yyyy':
        return date['day'] + '.' + str(date['month'])  + '.' + str(date['year'])
    if date_format == 'mm/dd/yyyy':
        return date['month'] + '/' + str(date['day'])  + '/' + str(date['year'])

def correct_dates(source_segment, target_segment, date_format):
    dates_source = find_dates(source_segment)
    dates_target = find_dates(target_segment)
    if len(dates_source) != len(dates_target):
        print(f'OSTRZEŻENIE: liczba dat w tłumaczeniu ({len(dates_target)}) nie zgadza się z liczbą dat w segmencie źródłowym ({len(dates_source)})')
    for source_date, target_date in zip(dates_source, dates_target):
        if source_date != target_date:
            print(f'OSTRZEŻENIE: data z segemtnu źródłowego {source_date} nie zgadza się z datą w segmencie z tłumaczenia {target_date}')
    if date_format != 'rrrr-mm-dd' and date_format != 'dd.mm.yyyy' and date_format != 'mm/dd/yyyy':
        return 'BŁĄD: nieobsługiwany format daty'
    for date in dates_target:
        target_segment = target_segment[:date['pos_start']] + build_date(date, date_format) + target_segment[date['pos_end']:]
    return target_segment


correct_dates('2021-04-07 2021/04/07 07.04.2021 04/07/2021', '2021-04-07 2021/04/07 07.04.2021 04/07/2021', 'mm/dd/yyyy')

'04/07/2021 04/07/2021 04/07/2021 04/07/2021'

Co jeszcze można zrobić? Zajmijmy się tagami XML. Z punktu widzenia tłumacza najlepiej byłoby, gdyby mógł przetłumaczyć segment źródłowy zawierający tagi XML na język docelowy zupełnie ignorując te tagi. Ponieważ jednak tagi muszą jakoś znaleźć się w segmencie docelowym, przydałaby się jakaś "magiczna różdżka", która przeniosłaby wszystkie tagi ze źródła do tłumaczenia na mniej więcej dobre miejsca. Spełnijmy marzenie tłumacza!

Rozważmy następujący algorytm: na wejściu mamy segment źródłowy zawierający tagi oraz segment docelowy bez tagów. Dokonujemy tokenizacji segmentu źródłowego tak, aby tagi były osobnymi tokenami. Następnie przeprowadźmy tokenizację segmentu docelowego. Gdy to jest gotowe, możemy zabrać się za przenoszenie (kopiowanie) tagów z segmentu źródłowego do docelowego.

![Transfer tagów](img/tagtransfer.png)

Gdzie w segmencie docelowym powinien trafić tag? Przede wszystkim pomiędzy tokeny - nie chcemy rozbijać słów tagami. Pytanie tylko, pomiędzy które tokeny? Jeśli sytuacja jest taka, jak powyżej, kiedy segment źródłowy i docelowy mają taką samą liczbę słów nie będących tagami, przenosimy tagi na odpowiadające pozycje w tłumaczeniu. Natomiast jeśli długość tłumaczenia jest inna niż źródła, należy obliczać te pozycje w sposób proporcjonalny - jeśli np. mamy tag w źródle na pozycji 3, a tłumaczenie jest dwa razy dłuższe niż źródło, tag powinien być przeniesiony do tłumaczenia na pozycję 6. W przypadku niecałkowitych wartości proporcji stosujemy zaokrąglenia.

### Ćwiczenie 5: Zaimplementuj opisany algorytm transferu tagów.

In [159]:
from nltk.tokenize import word_tokenize

def transfer_tags(source_segment, target_segment):
    tags_ranges = find_tags(source_segment)
    tags_positions = []
    for index, tag in enumerate(tags_ranges):
        tags_positions.append(len(word_tokenize(source_segment[:tag[0]])) - index * 3) # - index * 3 -> ignorowanie poprzedzających tagów (każdy to 3 słowa w word_tokenize) 
    tags_with_positions = list(zip([source_segment[tag_range[0]:tag_range[1]] for tag_range in tags_ranges], tags_positions))
    source_len = len(word_tokenize(source_segment)) - 3 * len(tags_with_positions)
    target_words = word_tokenize(target_segment)
    proportion = len(target_words) / source_len
    for index, (tag, position) in enumerate(tags_with_positions):
        target_words.insert(round(position * proportion) + index, tag)
    return ' '.join(target_words)

transfer_tags('<quote> I\'m Pooh </quote> - said Pooh', 'Jestem Puchatek - rzekł Puchatek')

'<quote> Jestem Puchatek </quote> - rzekł Puchatek'